In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
raw_data = pd.read_csv('epi_r.csv')

#Drop missing values
raw_data = raw_data.dropna()

# Drop rows with a rating of 0
raw_data = raw_data[(raw_data['rating'] != 0)]

In [3]:
from sklearn.svm import SVR
svr = SVR()
Y = raw_data.rating
raw_data = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'], 1)

# Replace the 1-5 rating system with 0 for 1-2.49 and 1 for 2.50-5.00
Y = np.where(Y >= 2.5, 1, 0)

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selects the 30 best features to determine rating
kbest = SelectKBest().fit(raw_data,Y)
result_kb = pd.DataFrame(kbest.score_func(raw_data,Y)[1], index=raw_data.columns)
result_kb = result_kb[0].sort_values().head(30)

# Store the values in a new dataframe
X = raw_data[result_kb.index.values.tolist()]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [ 93 105 163 183 195 213 221 225 226 234 243 288 294 308 319 358 381 394
 397 400 463 489 501 524 567 643 670] are constant.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [6]:
# Fit the svr model
svr.fit(X,Y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [7]:
# Print the accuracy of our test
svr.score(X, Y)

-0.5344132847362193

In [8]:
# Perform cross validation to determine repeatability
from sklearn.model_selection import cross_val_score
cross_val_score(svr, X, Y, cv=5)

array([-0.82276398, -0.70082102, -0.43008621, -0.50082588, -0.39526725])